In [5]:
from huggingface_hub import login 
login(token="hidden_token_id")

In [6]:
from huggingface_hub import HfApi, HfFolder, Repository

# Define the repository name and path
repo_name = "sentiment-analysis-roberta"  # Replace with your desired model name
repo_url = HfApi().create_repo(repo_name, exist_ok=True)

# Upload the model
api = HfApi()
api.upload_folder(
    folder_path="E:\git_hub\saved_fine_tuned_model", 
    repo_id="kai-pn/" + repo_name,  
    repo_type="model"
)
print(f"Model uploaded to {repo_url}")

model.safetensors: 100%|██████████| 499M/499M [00:31<00:00, 15.7MB/s]  


Model uploaded to https://huggingface.co/kai-pn/sentiment-analysis-roberta


### Testing

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

model_name = "kai-pn/sentiment-analysis-roberta" 
model = RobertaForSequenceClassification.from_pretrained(model_name)
tokenizer = RobertaTokenizer.from_pretrained(model_name)

texts = ["Ok. not too excellent but acceptable for its cost."]

inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():  
    outputs = model(**inputs)  
    predictions = outputs.logits.argmax(dim=-1)  

label_mapping_reverse = {0: 'NEG', 1: 'NEU', 2: 'POS'}
results = [label_mapping_reverse[pred.item()] for pred in predictions] 

for text, sentiment in zip(texts, results):
    print(f"Text: {text}\nPredicted Sentiment: {sentiment}\n")

Text: Ok. not too excellent but acceptable for its cost.
Predicted Sentiment: NEU



In [ ]:
def load_model_and_tokenizer(model_path='kai-pn/sentiment-analysis-roberta'):
    try:
        model = RobertaForSequenceClassification.from_pretrained(model_path)
        tokenizer = RobertaTokenizer.from_pretrained(model_path)

        model.eval()

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        print(f"Model and tokenizer successfully loaded from {model_path}")
        print(f"Model configured with {model.config.num_labels} labels.")   

        return model, tokenizer, device

    except Exception as e:
        raise ValueError(f"Error loading model or tokenizer from {model_path}: {e}")

def predict(texts, model, tokenizer, device):
    """Predict sentiments for a list of texts."""

    inputs = tokenizer(texts, truncation=True, padding=True, return_tensors='pt').to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()  # Move outputs to CPU for further processing
    
    label_mapping_reverse = {0: 'NEG', 1: 'NEU', 2: 'POS'}
    return [label_mapping_reverse[pred] for pred in predictions]

if __name__ == "__main__":
    model, tokenizer, device = load_model_and_tokenizer('kai-pn/sentiment-analysis-roberta')
    
    sample_texts = [
        "The product is excellent, works as expected!",
        "Terrible experience, I want my money back.",
        "It's okay, not great but not bad either."
    ]
    predictions = predict(sample_texts, model, tokenizer, device)
    for text, label in zip(sample_texts, predictions):
        print(f"Text: {text}\nPredicted Sentiment: {label}\n")

Model and tokenizer successfully loaded from kai-pn/sentiment-analysis-roberta
Model configured with 3 labels.
Text: The product is excellent, works as expected!
Predicted Sentiment: POS

Text: Terrible experience, I want my money back.
Predicted Sentiment: NEG

Text: It's okay, not great but not bad either.
Predicted Sentiment: NEU

